In [ ]:
# @title 0. Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

print("✅ เชื่อมต่อเรียบร้อย!")

In [ ]:
# @title 1. Setup Environment & Install DeepSeek-R1

!curl -fsSL https://ollama.com/install.sh | sh
!pip install pandas tqdm

import subprocess
import time
import os

# Start Ollama Server
print("Starting Ollama Server...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)

MODEL_NAME = "deepseek-r1:latest"
print(f"Downloading model: {MODEL_NAME}...")
!ollama pull {MODEL_NAME}

print("✅ Setup Complete!")

In [ ]:
# @title 2. Configuration
import pandas as pd
import requests
import json
import os
import re
from tqdm import tqdm

TARGET_FILE_NAME = "sentiment_ner.csv"

INPUT_DIR = "/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Dataset"
OUTPUT_DIR = "/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/Deepseek-r1_temp1_Topp0.07/sentiment_ner"
INPUT_COLUMN = "Perturbed"
TARGET_MODEL = "deepseek-r1:latest"
OUTPUT_FILENAME_PREFIX = "Result_Deepseek-r1_temp1_"

print(f"✅ Config Saved: เตรียมรันไฟล์ {TARGET_FILE_NAME} ด้วยโมเดล {TARGET_MODEL}")

In [ ]:
# @title 3. Helper Functions

def clean_deepseek_response(text):
    if not text: return "Error"

    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)

    text_lower = text.lower()
    if "positive" in text_lower: return "Positive"
    elif "negative" in text_lower: return "Negative"
    elif "neutral" in text_lower: return "Neutral"
    else: return text.strip()

def create_prompt(sentence):
    system_part = """You are an assistant that classifies the sentiment of the
message into positive, negative, and neutral. Given below is an example
of the sentiment analysis task.

Sentence: I had a bad experience
Sentiment: Negative"""

    user_part = f"""What is the sentiment of the following sentence? Limit your
answer to only one of these options: Positive, Negative, or Neutral.
{sentence}"""

    return [
        {"role": "system", "content": system_part},
        {"role": "user", "content": user_part}
    ]

def query_ollama(model, messages):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model, "messages": messages, "stream": False,
        "options": {"temperature": 1.0,
                    "top_p": 0.07}
    }
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            return response.json()['message']['content']
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

print("✅ ระบบพร้อมทำงาน")

In [ ]:
# @title 4. Start Processing
NUM_RUNS = 3

input_path = os.path.join(INPUT_DIR, TARGET_FILE_NAME)
target_output_filename = f"{OUTPUT_FILENAME_PREFIX}{TARGET_FILE_NAME}"
output_path = os.path.join(OUTPUT_DIR, f"{OUTPUT_FILENAME_PREFIX}{TARGET_FILE_NAME}")

print(f"🚀 Processing: {TARGET_FILE_NAME}")
print(f"🔄 Mode: Multiple Runs ({NUM_RUNS} times per sentence)")
print(f"📂 Reading from: {input_path}")

if not os.path.exists(input_path):
    print(f"❌ Error: ไม่พบไฟล์ '{input_path}'")
else:
    try:
        df = pd.read_csv(input_path)
        if INPUT_COLUMN not in df.columns:
            print(f"❌ Error: ไม่พบคอลัมน์ชื่อ '{INPUT_COLUMN}' ในไฟล์ CSV")
        else:
            all_predictions = {f"Prediction{i}": [] for i in range(1, NUM_RUNS + 1)}

            # --- เริ่มวนลูป ---
            for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating"):
                sentence = str(row[INPUT_COLUMN])
                prompt_messages = create_prompt(sentence)
                for i in range(1, NUM_RUNS + 1):
                    raw_res = query_ollama(TARGET_MODEL, prompt_messages)
                    final_pred = clean_deepseek_response(raw_res)

                    # เก็บลง list ของรอบนั้นๆ
                    all_predictions[f"Prediction{i}"].append(final_pred)

            ref_col_name = 'Expected_answer'

            if ref_col_name in df.columns:
                insert_loc_start = df.columns.get_loc(ref_col_name) + 1
            else:
                print(f"⚠️ Warning: ไม่พบ '{ref_col_name}'.. จะบันทึกไว้ท้ายสุด")
                insert_loc_start = len(df.columns)

            # วนลูปเอาข้อมูลใส่ตาราง
            for i in range(1, NUM_RUNS + 1):
                col_name = f"Prediction{i}"

                if col_name in df.columns:
                    df.drop(columns=[col_name], inplace=True)

                current_insert_loc = insert_loc_start + (i - 1)

                if ref_col_name not in df.columns:
                    df[col_name] = all_predictions[col_name]
                else:
                    # แทรกคอลัมน์
                    if current_insert_loc > len(df.columns):
                        df[col_name] = all_predictions[col_name]
                    else:
                        df.insert(current_insert_loc, col_name, all_predictions[col_name])

            # --- บันทึกไฟล์ ---
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"\n✅ เสร็จสมบูรณ์! บันทึกผลลัพธ์ที่มี 3 Predictions ที่: {output_path}")

    except Exception as e:
        print(f"\n❌ Critical Error: {e}")